# Migration des Données Médicales vers MongoDB

## Objectif
Ce notebook a pour but de démontrer les étapes nécessaires pour migrer un fichier de données médicales au format CSV vers une base de données MongoDB, avec les étapes suivantes :
1. Chargement et validation des données.
2. Nettoyage et export des données.
3. Insertion des données dans MongoDB.
4. Vérification des documents insérés dans la base de données.

## Fichiers requis
- **healthcare_dataset.csv** : Fichier source contenant les données médicales. Ce fichier doit être placé dans le répertoire `data/raw/`.

## Résultat attendu
Les données nettoyées seront enregistrées dans `data/processed/healthcare_data_cleaned.csv` et insérées dans une base MongoDB locale.


In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd  # Pour la manipulation des données
import os  # Pour interagir avec le système de fichiers
from loguru import logger  # Pour gérer les logs
from pymongo import MongoClient  # Pour se connecter à MongoDB
from pathlib import Path  # Pour gérer les chemins de fichiers et répertoires
import sys  # Pour modifier le chemin d'import des modules
import kagglehub  # Pour télécharger les datasets Kaggle
from skimpy import skim  # Résumés statistiques enrichis pour les DataFrames
import warnings   # Suppression des avertissements inutiles

# --- Configuration des Affichages pandas ---
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes d'un DataFrame
pd.set_option('display.max_rows', 100)      # Affiche jusqu'à 100 lignes dans un DataFrame
pd.set_option('display.width', 400)         # Évite les retours à la ligne inutiles

# --- Suppression des Avertissements Inutiles ---
warnings.filterwarnings("ignore")

# Configuration des logs
# Les logs sont enregistrés dans un fichier avec une rotation automatique pour limiter la taille
logger.add("../logs/migration.log", rotation="500 MB")
logger.info("Démarrage du notebook")

# Définir le répertoire de base pour un notebook ou un script
# Si on est dans un notebook, on remonte au répertoire parent
NOTEBOOK_PATH = Path(os.getcwd())
if NOTEBOOK_PATH.name == "notebooks":
    BASE_DIR = NOTEBOOK_PATH.parent
else:
    BASE_DIR = NOTEBOOK_PATH

# Ajouter le chemin du répertoire 'src' au sys.path
sys.path.append(str(BASE_DIR / "src"))

# Définir les chemins des répertoires à partir du répertoire de base
RAW_DIR = BASE_DIR / "data/raw"  # Répertoire pour les fichiers bruts
PROCESSED_DIR = BASE_DIR / "data/processed"  # Répertoire pour les fichiers nettoyés
SCRIPTS_DIR = BASE_DIR / "scripts"  # Répertoire des scripts

# --- Fonction pour configurer et loguer les répertoires ---
def setup_and_log_directories(base_dir, raw_dir, processed_dir, scripts_dir):
    """
    Configure les répertoires pour les fichiers bruts, nettoyés, et les scripts.
    Crée les répertoires si nécessaire et logue leur statut en chemins relatifs avec des descriptions.

    Returns:
        dict: Chemins relatifs et absolus des répertoires.
    """
    try:
        # Convertir les chemins en chemins relatifs
        base_dir_relative = base_dir.name  # Nom du répertoire racine
        raw_dir_relative = raw_dir.relative_to(base_dir)
        processed_dir_relative = processed_dir.relative_to(base_dir)
        scripts_dir_relative = scripts_dir.relative_to(base_dir)

        # Log du répertoire racine
        logger.info(f"Répertoire racine pour le projet défini : {base_dir_relative}")

        # Crée ou vérifie le répertoire des fichiers bruts
        raw_dir.mkdir(parents=True, exist_ok=True)
        logger.info(f"Répertoire pour les fichiers bruts (RAW_DIR) configuré : {base_dir_relative}\\{raw_dir_relative}")

        # Crée ou vérifie le répertoire des fichiers nettoyés
        processed_dir.mkdir(parents=True, exist_ok=True)
        logger.info(f"Répertoire pour les fichiers nettoyés (PROCESSED_DIR) configuré : {base_dir_relative}\\{processed_dir_relative}")

        # Crée ou vérifie le répertoire des scripts
        scripts_dir.mkdir(parents=True, exist_ok=True)
        logger.info(f"Répertoire pour les scripts (SCRIPTS_DIR) configuré : {base_dir_relative}\\{scripts_dir_relative}")

        # Retourner les chemins relatifs et absolus
        return {
            "base_dir": base_dir,
            "raw_dir": {"absolute": raw_dir, "relative": raw_dir_relative},
            "processed_dir": {"absolute": processed_dir, "relative": processed_dir_relative},
            "scripts_dir": {"absolute": scripts_dir, "relative": scripts_dir_relative},
        }

    except Exception as e:
        # Log des erreurs en cas de problème
        logger.error(f"Erreur lors de la configuration des répertoires : {e}")
        raise

# Configuration des répertoires et récupération des chemins relatifs et absolus
directories = setup_and_log_directories(BASE_DIR, RAW_DIR, PROCESSED_DIR, SCRIPTS_DIR)

logger.info("Répertoires vérifiés et créés avec succès.")

2025-01-21 09:50:39.499 | INFO     | __main__:<module>:23 - Démarrage du notebook
2025-01-21 09:50:39.501 | INFO     | __main__:setup_and_log_directories:58 - Répertoire racine pour le projet défini : migration-donnees-mongodb
2025-01-21 09:50:39.502 | INFO     | __main__:setup_and_log_directories:62 - Répertoire pour les fichiers bruts (RAW_DIR) configuré : migration-donnees-mongodb\data\raw
2025-01-21 09:50:39.504 | INFO     | __main__:setup_and_log_directories:66 - Répertoire pour les fichiers nettoyés (PROCESSED_DIR) configuré : migration-donnees-mongodb\data\processed
2025-01-21 09:50:39.505 | INFO     | __main__:setup_and_log_directories:70 - Répertoire pour les scripts (SCRIPTS_DIR) configuré : migration-donnees-mongodb\scripts
2025-01-21 09:50:39.506 | INFO     | __main__:<module>:88 - Répertoires vérifiés et créés avec succès.


In [2]:
# -------------------------------------------
# Téléchargement et Chargement des Données
# -------------------------------------------

# Téléchargement du dataset depuis Kaggle
try:
    # Télécharge le dataset en utilisant KaggleHub
    dataset_path = kagglehub.dataset_download("prasad22/healthcare-dataset")
    logger.info(f"Dataset téléchargé avec succès")
except Exception as e:
    # Gère les erreurs de téléchargement et les logue
    logger.error(f"Erreur lors du téléchargement du dataset : {e}")
    raise  # Arrête l'exécution en cas d'échec

# Étape 2 : Lister les fichiers disponibles dans le dossier téléchargé
try:
    # Liste tous les fichiers dans le répertoire du dataset
    files = os.listdir(dataset_path)
    logger.info(f"Fichiers disponibles dans le dataset : {files}")
except Exception as e:
    # Log en cas d'échec de la lecture du répertoire
    logger.error(f"Erreur lors de la lecture des fichiers dans le répertoire {dataset_path} : {e}")
    raise

# Étape 3 : Chargement d'un fichier spécifique
# Nom du fichier attendu
dataset_file = Path(dataset_path) / "healthcare_dataset.csv"
try:
    # Charge le fichier CSV en mémoire en tant que DataFrame pandas
    data = pd.read_csv(dataset_file)
    logger.info(f"Fichier '{dataset_file.name}' chargé avec succès. "
                f"Nombre de lignes : {data.shape[0]}. Nombre de colonnes : {data.shape[1]}.")
except FileNotFoundError:
    # Log en cas de fichier manquant
    logger.error(f"Le fichier spécifié '{dataset_file.name}' est introuvable dans le dataset.")
    raise
except Exception as e:
    # Log pour tout autre type d'erreur
    logger.error(f"Erreur lors du chargement du fichier '{dataset_file.name}' : {e}")
    raise


2025-01-21 09:50:40.178 | INFO     | __main__:<module>:9 - Dataset téléchargé avec succès
2025-01-21 09:50:40.179 | INFO     | __main__:<module>:19 - Fichiers disponibles dans le dataset : ['healthcare_dataset.csv']
2025-01-21 09:50:40.281 | INFO     | __main__:<module>:31 - Fichier 'healthcare_dataset.csv' chargé avec succès. Nombre de lignes : 55500. Nombre de colonnes : 15.


In [3]:
# 🚀 Exportation des DataFrames en CSV avec encodage UTF-8
logger.info("=" * 60)
logger.info("Début de l'exportation des DataFrames en fichiers CSV")
logger.info("-" * 10)

def export_dataframes_to_csv(dataframes, output_directory, base_dir):
    """
    Exporte chaque DataFrame en fichier CSV dans le répertoire spécifié avec encodage UTF-8.

    Args:
        dataframes (dict): Dictionnaire de DataFrames à exporter.
        output_directory (Path): Répertoire où sauvegarder les fichiers CSV.
        base_dir (Path): Répertoire racine pour générer des chemins relatifs dans les logs.
    """
    if not isinstance(output_directory, Path):
        logger.error(f"Le chemin '{output_directory}' n'est pas une instance de Path.")
        return

    if not output_directory.exists():
        logger.warning(f"Le répertoire '{output_directory}' n'existe pas. Création du répertoire.")
        output_directory.mkdir(parents=True, exist_ok=True)

    for name, df in dataframes.items():
        if not isinstance(df, pd.DataFrame):
            logger.error(f"'{name}' n'est pas un DataFrame valide. Exportation ignorée.")
            continue

        export_path = output_directory / f"{name}.csv"
        try:
            # Exporter le DataFrame en CSV
            df.to_csv(export_path, index=False, encoding='utf-8')

            # Convertir le chemin absolu en chemin relatif pour les logs
            relative_path = export_path.relative_to(base_dir)
            logger.info(f"✅ DataFrame '{name}' exporté avec succès : {relative_path}")
        except Exception as e:
            logger.error(f"❌ Erreur lors de l'exportation du DataFrame '{name}': {e}")

 
healthcare_dataset = {"healthcare_data": data}
export_dataframes_to_csv(healthcare_dataset, directories["raw_dir"]["absolute"], BASE_DIR)

logger.info("-" * 10)
logger.info("Fin de l'exportation des DataFrames en fichiers CSV")
logger.info("=" * 60)

2025-01-21 09:50:40.294 | INFO     | __main__:<module>:2 - ============================================================
2025-01-21 09:50:40.296 | INFO     | __main__:<module>:3 - Début de l'exportation des DataFrames en fichiers CSV
2025-01-21 09:50:40.297 | INFO     | __main__:<module>:4 - ----------
2025-01-21 09:50:40.517 | INFO     | __main__:export_dataframes_to_csv:35 - ✅ DataFrame 'healthcare_data' exporté avec succès : data\raw\healthcare_data.csv
2025-01-21 09:50:40.518 | INFO     | __main__:<module>:43 - ----------
2025-01-21 09:50:40.518 | INFO     | __main__:<module>:44 - Fin de l'exportation des DataFrames en fichiers CSV
2025-01-21 09:50:40.519 | INFO     | __main__:<module>:45 - ============================================================


In [4]:
# Aperçu des données
# Afficher les 5 premières lignes pour un aperçu rapide du contenu
logger.info(f"Aperçu des données : \n{data.head()}\n")

# Résumé statistique avec skim
# Fournit une vue d'ensemble rapide des colonnes, types, valeurs manquantes, etc.
logger.info(f"Résumé pour le dataset 'healthcare_dataset':")
skim(data)  # Assurez-vous que skimpy est installé : pip install skimpy

# Séparateur visuel dans les logs
logger.info(f"{'=' * 60}")

# Types des colonnes
# Affiche les types des colonnes pour identifier d'éventuels problèmes
logger.info(f"📊 Types des colonnes pour le DataFrame 'healthcare'")
logger.info(f"{'-' * 60}")

# Boucle pour afficher les détails de chaque colonne
# Cela inclut le type de données pandas et les types Python uniques rencontrés
for col in data.columns:
    col_type = data[col].dtype  # Type de colonne selon pandas (int64, object, etc.)
    unique_types = data[col].apply(lambda x: type(x).__name__).unique()  # Types Python rencontrés
    logger.info(f"🔹 {col} : {col_type} | Valeurs : {', '.join(unique_types)}")

# Séparateur final
logger.info(f"{'=' * 60}\n")


2025-01-21 09:50:40.531 | INFO     | __main__:<module>:3 - Aperçu des données : 
            Name  Age  Gender Blood Type Medical Condition Date of Admission            Doctor                    Hospital Insurance Provider  Billing Amount  Room Number Admission Type Discharge Date   Medication  Test Results
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31     Matthew Smith             Sons and Miller         Blue Cross    18856.281306          328         Urgent     2024-02-02  Paracetamol        Normal
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   Samantha Davies                     Kim Inc           Medicare    33643.327287          265      Emergency     2019-08-26    Ibuprofen  Inconclusive
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22  Tiffany Mitchell                    Cook PLC              Aetna    27955.096079          205      Emergency     2022-10-07      Aspirin        Normal
3   and

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 55500  │ │ string      │ 12    │                                                          │
│ │ Number of columns │ 15     │ │ int32       │ 2     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name       ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50    ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ Age               │   0 │      0 │   51.54 │   19.6 │     13 │     35 │     52 │     68 │     89 │ ▅▇▇▇▇▅  │  │
│ │ Billing Amount    │   0 │      0 │   25540 │  14210 │  -2008 │  13240 │  25540 │  37820 │  52760 │ ▅▇▇▇▇▆  │  │
│ │ Room Number       │   0 │      0 │   301.1 │  115.2 │    101 │    202 │    302 │    401 │    500 │ ▇▇▇▇▇▇  │  │
│ └───────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴────────┴────────┴────────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                        ┃ NA     ┃ NA %      ┃ words per row             ┃ total words          ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ Name                               │      0 │         0 │                         2 │               113390 │  │
│ │ Gender                             │      0 │         0 │                         1 │                55500 │  │
│ │ Blood Type                         │      0 │         0 │                         1 │                55500 │  │
│ │ Medical Condition                  │      0 │         0 │                         1 │                55500 │  │
│ │ Date of Admission                  │      0 │         0 │                         1 │                55500 │  │
│ │ Doctor                             │      0 │         0 │                         2 │               113516 │  │
│ │ Hospital                           │      0 │         0 │                       2.4 │               133009 │  │
│ │ Insurance Provider                 │      0 │         0 │                       1.2 │                66559 │  │
│ │ Admission Type                     │      0 │         0 │                         1 │                55500 │  │
│ │ Discharge Date                     │      0 │         0 │                         1 │                55500 │  │
│ │ Medication                         │      0 │         0 │                         1 │                55500 │  │
│ │ Test Results                       │      0 │         0 │                         1 │                55500 │  │
│ └────────────────────────────────────┴────────┴───────────┴───────────────────────────┴──────────────────────┘  │
╰────────────────────────────────────────────────────── 

2025-01-21 09:50:40.860 | INFO     | __main__:<module>:11 - ============================================================
2025-01-21 09:50:40.861 | INFO     | __main__:<module>:15 - 📊 Types des colonnes pour le DataFrame 'healthcare'
2025-01-21 09:50:40.862 | INFO     | __main__:<module>:16 - ------------------------------------------------------------
2025-01-21 09:50:40.873 | INFO     | __main__:<module>:23 - 🔹 Name : object | Valeurs : str
2025-01-21 09:50:40.884 | INFO     | __main__:<module>:23 - 🔹 Age : int64 | Valeurs : int
2025-01-21 09:50:40.895 | INFO     | __main__:<module>:23 - 🔹 Gender : object | Valeurs : str
2025-01-21 09:50:40.906 | INFO     | __main__:<module>:23 - 🔹 Blood Type : object | Valeurs : str
2025-01-21 09:50:40.916 | INFO     | __main__:<module>:23 - 🔹 Medical Condition : object | Valeurs : str
2025-01-21 09:50:40.926 | INFO     | __main__:<module>:23 - 🔹 Date of Admission : object | Valeurs : str
2025-01-21 09:50:40.937 | INFO     | __main__:<module>:23 - 🔹 

In [5]:
# Identification des doublons
data['is_duplicate'] = data.duplicated(keep=False)

# Filtrer les lignes marquées comme doublons
duplicates = data[data['is_duplicate']].copy()

# Ajouter une colonne 'is_primary' pour identifier la première occurrence
duplicates['is_primary'] = ~duplicates.duplicated(keep='first')

# Normaliser les colonnes avant le traitement
columns_to_normalize = [col for col in data.columns if col not in ['is_duplicate', 'is_primary']]
for col in columns_to_normalize:
    if data[col].dtype == 'object':  # Normaliser uniquement les colonnes de type texte
        duplicates[col] = duplicates[col].str.strip().str.lower()  # Supprimer les espaces et uniformiser les majuscules

# Ajouter un identifiant temporaire unique pour chaque ligne
duplicates = duplicates.reset_index(drop=False).rename(columns={'index': 'temp_id'})

# Séparer les lignes principales et les doublons
primary_rows = duplicates[duplicates['is_primary']].copy()
duplicate_rows = duplicates[~duplicates['is_primary']].copy()

# Ajouter un identifiant unique pour les groupes de doublons
primary_rows['group_id'] = range(1, len(primary_rows) + 1)

# Associer les doublons à leurs lignes principales
duplicates_with_group = duplicate_rows.merge(
    primary_rows[columns_to_normalize + ['group_id']], 
    how='left', 
    on=columns_to_normalize, 
    suffixes=('', '_primary')
)

# Vérification des colonnes générées après le merge
generated_columns = duplicates_with_group.columns
logger.info(f"Colonnes générées après le merge : {generated_columns}")

# Comparer chaque colonne entre les lignes principales et leurs doublons
diff_columns = []  # Initialisation de la liste des colonnes de différences
for col in columns_to_normalize:
    primary_col = f"{col}_primary"
    if primary_col in duplicates_with_group.columns:
        diff_col = f"{col}_diff"
        duplicates_with_group[diff_col] = duplicates_with_group[col] != duplicates_with_group[primary_col]
        diff_columns.append(diff_col)  # Ajouter à la liste des colonnes de différences
    else:
        logger.warning(f"La colonne '{primary_col}' n'existe pas. Comparaison ignorée.")

# Ajouter la liste des groupes concernés par des doublons
if not duplicates_with_group.empty:  # Vérifie si le DataFrame n'est pas vide
    group_ids = duplicates_with_group['group_id'].unique()  # Extraire les identifiants des groupes
    logger.info(f"Groupes concernés par des doublons : {group_ids.tolist()}")
else:
    logger.info("Aucun groupe concerné par des doublons.")


# Résultat final pour analyse
logger.info(f"Groupes de doublons avec colonnes de différences : \n{duplicates_with_group.head()}")
logger.info(f"Colonnes avec des différences détectées : {diff_columns}")


2025-01-21 09:50:41.102 | INFO     | __main__:<module>:36 - Colonnes générées après le merge : Index(['temp_id', 'Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider', 'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date', 'Medication', 'Test Results', 'is_duplicate', 'is_primary', 'group_id'], dtype='object')
2025-01-21 09:50:41.104 | WARNING  | __main__:<module>:47 - La colonne 'Name_primary' n'existe pas. Comparaison ignorée.
2025-01-21 09:50:41.105 | WARNING  | __main__:<module>:47 - La colonne 'Age_primary' n'existe pas. Comparaison ignorée.
2025-01-21 09:50:41.106 | WARNING  | __main__:<module>:47 - La colonne 'Gender_primary' n'existe pas. Comparaison ignorée.
2025-01-21 09:50:41.107 | WARNING  | __main__:<module>:47 - La colonne 'Blood Type_primary' n'existe pas. Comparaison ignorée.
2025-01-21 09:50:41.109 | WARNING  | __main__:<module>:47 - La colonne 'Medical Condition_primary' n'existe pas. 

In [6]:
# Liste des groupes à analyser
groups_to_check = [81, 21, 279, 61, 211, 383, 112, 196, 231, 55, 309, 508, 172, 384, 387, 202, 450, 92, 481, 432, 223, 307]  # Remplacez par les identifiants des groupes que vous souhaitez analyser

# Filtrer les lignes principales et les doublons pour les groupes sélectionnés
groups_primary = primary_rows[primary_rows['group_id'].isin(groups_to_check)]
groups_duplicates = duplicates_with_group[duplicates_with_group['group_id'].isin(groups_to_check)]

# Combiner les lignes principales et les doublons pour analyse
groups_to_inspect = pd.concat([groups_primary, groups_duplicates])

# Trier les résultats par 'group_id' pour regrouper les lignes par groupe
groups_to_inspect = groups_to_inspect.sort_values(by=['group_id', 'is_primary'], ascending=[True, False])

# Afficher le contenu des groupes pour validation
logger.info(f"Contenu complet des groupes triés par 'group_id' : \n{groups_to_inspect}")


2025-01-21 09:50:41.143 | INFO     | __main__:<module>:15 - Contenu complet des groupes triés par 'group_id' : 
     temp_id                 Name  Age  Gender Blood Type Medical Condition Date of Admission                 Doctor                      Hospital Insurance Provider  Billing Amount  Room Number Admission Type Discharge Date   Medication  Test Results  is_duplicate  is_primary  group_id
20      1336     kimberly vasquez   26    male         a-           obesity        2023-10-23       jennifer bennett                     cowan inc   unitedhealthcare    38142.109678          313         urgent     2023-11-18   penicillin      abnormal          True        True        21
1      50040     kimberly vasquez   26    male         a-           obesity        2023-10-23       jennifer bennett                     cowan inc   unitedhealthcare    38142.109678          313         urgent     2023-11-18   penicillin      abnormal          True       False        21
54      3960      regina

In [7]:
# Identification des doublons
data['is_duplicate'] = data.duplicated(keep=False)

# Filtrer les lignes marquées comme doublons
duplicates = data[data['is_duplicate']].copy()

# Ajouter une colonne 'is_primary' pour identifier la première occurrence
duplicates['is_primary'] = ~duplicates.duplicated(keep='first')

# Rattacher la colonne 'is_primary' à l'ensemble du DataFrame
data['is_primary'] = data.index.isin(duplicates.index[duplicates['is_primary']])

# Garder uniquement les lignes principales et les lignes sans doublons
cleaned_data = data[data['is_primary'] | ~data['is_duplicate']].copy()

# Supprimer les colonnes ajoutées pour l’analyse des doublons
columns_to_remove = ['is_primary', 'is_duplicate', 'group_id']
cleaned_data.drop(columns=columns_to_remove, inplace=True, errors='ignore')

# Afficher le résultat final
logger.info(f"Nombre de lignes après suppression des doublons : {cleaned_data.shape[0]}")
logger.info(f"Aperçu des données nettoyées : \n{cleaned_data.head()}")


2025-01-21 09:50:41.216 | INFO     | __main__:<module>:21 - Nombre de lignes après suppression des doublons : 54966
2025-01-21 09:50:41.219 | INFO     | __main__:<module>:22 - Aperçu des données nettoyées : 
            Name  Age  Gender Blood Type Medical Condition Date of Admission            Doctor                    Hospital Insurance Provider  Billing Amount  Room Number Admission Type Discharge Date   Medication  Test Results
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31     Matthew Smith             Sons and Miller         Blue Cross    18856.281306          328         Urgent     2024-02-02  Paracetamol        Normal
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   Samantha Davies                     Kim Inc           Medicare    33643.327287          265      Emergency     2019-08-26    Ibuprofen  Inconclusive
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22  Tiffany Mitchell              

In [8]:
# --- Analyse avant transformation ---

# Analyse initiale des colonnes et des types
logger.info("Analyse initiale des données :")
logger.info(f"Colonnes disponibles : {list(cleaned_data.columns)}")
logger.info(f"Types des colonnes avant transformation : \n{cleaned_data.dtypes}")

# Statistiques descriptives pour les colonnes de date et de texte
columns_to_analyze = ['Date of Admission', 'Discharge Date', 'Doctor', 'Hospital']
for col in columns_to_analyze:
    if col in cleaned_data.columns:
        logger.info(f"Statistiques pour '{col}': \n{cleaned_data[col].describe(include='all')}")

# Validation des colonnes catégoriques avant transformation
expected_gender = {'Male', 'Female'}
unexpected_gender = set(cleaned_data['Gender'].unique()) - expected_gender
logger.info(f"Valeurs initiales pour 'Gender' : {set(cleaned_data['Gender'].unique())}")
if unexpected_gender:
    logger.warning(f"Valeurs inattendues trouvées dans 'Gender' : {unexpected_gender}")

expected_blood_types = {'A+', 'A-', 'B+', 'B-', 'O+', 'O-', 'AB+', 'AB-'}
unexpected_blood_types = set(cleaned_data['Blood Type'].unique()) - expected_blood_types
logger.info(f"Valeurs initiales pour 'Blood Type' : {set(cleaned_data['Blood Type'].unique())}")
if unexpected_blood_types:
    logger.warning(f"Valeurs inattendues trouvées dans 'Blood Type' : {unexpected_blood_types}")


2025-01-21 09:50:41.230 | INFO     | __main__:<module>:4 - Analyse initiale des données :
2025-01-21 09:50:41.232 | INFO     | __main__:<module>:5 - Colonnes disponibles : ['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition', 'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider', 'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date', 'Medication', 'Test Results']
2025-01-21 09:50:41.234 | INFO     | __main__:<module>:6 - Types des colonnes avant transformation : 
Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object
2025-01-21 09:50:41.239 | INFO     | 

In [9]:
# --- Transformation des données ---

# Transformation des colonnes de dates
columns_to_convert = ['Date of Admission', 'Discharge Date']
for col in columns_to_convert:
    if col in cleaned_data.columns:
        cleaned_data[col] = pd.to_datetime(cleaned_data[col], errors='coerce')
logger.info("Les colonnes de dates ont été converties en type datetime.")

# Normalisation des colonnes de texte
columns_to_normalize = ['Name', 'Doctor', 'Hospital']
for col in columns_to_normalize:
    if col in cleaned_data.columns:
        cleaned_data[col] = cleaned_data[col].str.strip().str.title()
logger.info("Les colonnes de texte ont été normalisées (espaces supprimés, titres formatés).")


2025-01-21 09:50:41.312 | INFO     | __main__:<module>:8 - Les colonnes de dates ont été converties en type datetime.


2025-01-21 09:50:41.354 | INFO     | __main__:<module>:15 - Les colonnes de texte ont été normalisées (espaces supprimés, titres formatés).


In [10]:
# --- Analyse après transformation ---

# Analyse après transformation
logger.info("Analyse après transformation des données :")
logger.info(f"Types des colonnes après transformation : \n{cleaned_data.dtypes}")

# Comparaison des colonnes de dates
for col in columns_to_convert:
    if col in cleaned_data.columns:
        logger.info(f"Exemple de valeurs après conversion pour '{col}': \n{cleaned_data[col].head()}")

# Vérification des colonnes normalisées
for col in columns_to_normalize:
    if col in cleaned_data.columns:
        logger.info(f"Exemple de valeurs après normalisation pour '{col}': \n{cleaned_data[col].head()}")

2025-01-21 09:50:41.363 | INFO     | __main__:<module>:4 - Analyse après transformation des données :
2025-01-21 09:50:41.365 | INFO     | __main__:<module>:5 - Types des colonnes après transformation : 
Name                          object
Age                            int64
Gender                        object
Blood Type                    object
Medical Condition             object
Date of Admission     datetime64[ns]
Doctor                        object
Hospital                      object
Insurance Provider            object
Billing Amount               float64
Room Number                    int64
Admission Type                object
Discharge Date        datetime64[ns]
Medication                    object
Test Results                  object
dtype: object
2025-01-21 09:50:41.367 | INFO     | __main__:<module>:10 - Exemple de valeurs après conversion pour 'Date of Admission': 
0   2024-01-31
1   2019-08-20
2   2022-09-22
3   2020-11-18
4   2022-09-19
Name: Date of Admission, dtyp

In [20]:
# Afficher le résultat final
logger.info(f"Nombre de lignes après suppression des doublons : {cleaned_data.shape[0]}")
logger.info(f"Aperçu des données nettoyées : \n{cleaned_data.head()}")

2025-01-21 10:03:34.171 | INFO     | __main__:<module>:2 - Nombre de lignes après suppression des doublons : 54966
2025-01-21 10:03:34.175 | INFO     | __main__:<module>:3 - Aperçu des données nettoyées : 
            Name  Age  Gender Blood Type Medical Condition Date of Admission            Doctor                    Hospital Insurance Provider  Billing Amount  Room Number Admission Type Discharge Date   Medication  Test Results
0  Bobby Jackson   30    Male         B-            Cancer        2024-01-31     Matthew Smith             Sons And Miller         Blue Cross    18856.281306          328         Urgent     2024-02-02  Paracetamol        Normal
1   Leslie Terry   62    Male         A+           Obesity        2019-08-20   Samantha Davies                     Kim Inc           Medicare    33643.327287          265      Emergency     2019-08-26    Ibuprofen  Inconclusive
2    Danny Smith   76  Female         A-           Obesity        2022-09-22  Tiffany Mitchell                

In [21]:
# 🚀 Exportation des DataFrames en CSV avec encodage UTF-8
logger.info("=" * 60)
logger.info("Début de l'exportation des DataFrames en fichiers CSV")
logger.info("-" * 10)

cleaned_dataframes = {"healthcare_data": cleaned_data}
export_dataframes_to_csv(cleaned_dataframes, directories["processed_dir"]["absolute"], BASE_DIR)

logger.info("-" * 10)
logger.info("Fin de l'exportation des DataFrames en fichiers CSV")
logger.info("=" * 60)


2025-01-21 10:03:48.067 | INFO     | __main__:<module>:2 - ============================================================
2025-01-21 10:03:48.068 | INFO     | __main__:<module>:3 - Début de l'exportation des DataFrames en fichiers CSV
2025-01-21 10:03:48.069 | INFO     | __main__:<module>:4 - ----------
2025-01-21 10:03:48.263 | INFO     | __main__:export_dataframes_to_csv:35 - ✅ DataFrame 'healthcare_data' exporté avec succès : data\processed\healthcare_data.csv
2025-01-21 10:03:48.264 | INFO     | __main__:<module>:9 - ----------
2025-01-21 10:03:48.265 | INFO     | __main__:<module>:10 - Fin de l'exportation des DataFrames en fichiers CSV
2025-01-21 10:03:48.266 | INFO     | __main__:<module>:11 - ============================================================


In [26]:
# Paramètres de connexion MongoDB
host = "localhost"  # Utilisez "localhost" si MongoDB est sur la même machine
port = 27017        # Port par défaut de MongoDB
username = "admin_user"  # Identifiant administrateur défini dans setup_users.py
password = "secure_password"  # Mot de passe administrateur défini dans setup_users.py
database_name = "healthcare_database_notebook"  # Nom de la base à utiliser
collection_name = "patients_data"  # Nom de la collection

# Chemin du fichier CSV nettoyé
processed_dir = directories["processed_dir"]["absolute"]
cleaned_data_path = processed_dir / "healthcare_data.csv"  # Chemin absolu basé sur directories


try:
    # Construction de la chaîne de connexion
    connection_string = f"mongodb://{username}:{password}@{host}:{port}/?authSource=admin"
    client = MongoClient(connection_string)  # Connexion au serveur MongoDB
    db = client[database_name]  # Connexion à la base de données
    collection = db[collection_name]  # Connexion à la collection
    logger.info(f"Connexion réussie à MongoDB sur {host}:{port} pour la base '{database_name}'.")

    # Tester la connexion
    logger.info("Test de la connexion à MongoDB...")
    db.command("ping")  # Commande pour vérifier la connectivité
    logger.success("Connexion validée avec succès.")

    # Supprimez les documents existants dans la collection
    logger.info("Suppression des données existantes dans la collection...")
    collection.delete_many({})  # Supprime tous les documents de la collection
    logger.success("Collection nettoyée avec succès.")

    # Charger les données nettoyées
    if not cleaned_data_path.exists():
        logger.error(f"Le fichier {cleaned_data_path} est introuvable.")
        raise FileNotFoundError(f"Fichier introuvable : {cleaned_data_path}")

    cleaned_data = pd.read_csv(cleaned_data_path)  # Charger les données dans un DataFrame
    logger.info(f"Fichier CSV chargé avec succès : {cleaned_data_path}")

    # Insérer les données dans MongoDB
    logger.info("Insertion des données dans MongoDB...")
    records = cleaned_data.to_dict(orient="records")  # Convertir les données en dictionnaire
    collection.insert_many(records)  # Insérer les données dans MongoDB
    logger.success(f"Données insérées avec succès dans la collection '{collection_name}'.")

    # Vérifiez l'insertion
    count = collection.count_documents({})
    logger.info(f"Nombre total de documents dans la collection '{collection_name}': {count}")

except Exception as e:
    logger.error(f"Erreur lors de la connexion ou du chargement des données : {e}")
    raise


2025-01-21 10:06:33.042 | INFO     | __main__:<module>:20 - Connexion réussie à MongoDB sur localhost:27017 pour la base 'healthcare_database_notebook'.
2025-01-21 10:06:33.044 | INFO     | __main__:<module>:23 - Test de la connexion à MongoDB...
2025-01-21 10:06:33.057 | SUCCESS  | __main__:<module>:25 - Connexion validée avec succès.
2025-01-21 10:06:33.059 | INFO     | __main__:<module>:28 - Suppression des données existantes dans la collection...
2025-01-21 10:06:33.061 | SUCCESS  | __main__:<module>:30 - Collection nettoyée avec succès.
2025-01-21 10:06:33.159 | INFO     | __main__:<module>:38 - Fichier CSV chargé avec succès : f:\1.Boulot\02_Openclassrooms\Github\migration-donnees-mongodb\data\processed\healthcare_data.csv
2025-01-21 10:06:33.160 | INFO     | __main__:<module>:41 - Insertion des données dans MongoDB...
2025-01-21 10:06:34.328 | SUCCESS  | __main__:<module>:44 - Données insérées avec succès dans la collection 'patients_data'.
2025-01-21 10:06:34.359 | INFO     | _

In [27]:
# Vérifiez les données insérées dans MongoDB
logger.info("Exemple de données insérées dans MongoDB :")
example_records = collection.find().limit(5)  # Récupère 5 documents pour vérification
for record in example_records:
    logger.info(record)


2025-01-21 10:06:39.760 | INFO     | __main__:<module>:2 - Exemple de données insérées dans MongoDB :
2025-01-21 10:06:39.762 | INFO     | __main__:<module>:5 - {'_id': ObjectId('678f6399dda3f9c34ab9fada'), 'Name': 'Bobby Jackson', 'Age': 30, 'Gender': 'Male', 'Blood Type': 'B-', 'Medical Condition': 'Cancer', 'Date of Admission': '2024-01-31', 'Doctor': 'Matthew Smith', 'Hospital': 'Sons And Miller', 'Insurance Provider': 'Blue Cross', 'Billing Amount': 18856.281305978155, 'Room Number': 328, 'Admission Type': 'Urgent', 'Discharge Date': '2024-02-02', 'Medication': 'Paracetamol', 'Test Results': 'Normal'}
2025-01-21 10:06:39.765 | INFO     | __main__:<module>:5 - {'_id': ObjectId('678f6399dda3f9c34ab9fadb'), 'Name': 'Leslie Terry', 'Age': 62, 'Gender': 'Male', 'Blood Type': 'A+', 'Medical Condition': 'Obesity', 'Date of Admission': '2019-08-20', 'Doctor': 'Samantha Davies', 'Hospital': 'Kim Inc', 'Insurance Provider': 'Medicare', 'Billing Amount': 33643.327286577885, 'Room Number': 26

In [14]:
import sys
import pkg_resources

def list_used_libraries():
    # Obtenir les modules actuellement importés
    imported_modules = set(sys.modules.keys())

    # Obtenir la liste des librairies installées
    installed_packages = {pkg.key: pkg.version for pkg in pkg_resources.working_set}

    # Identifier les librairies importées présentes dans les packages installés
    used_libraries = {
        module: installed_packages[module]
        for module in imported_modules
        if module in installed_packages
    }

    # Afficher les résultats
    print("Librairies utilisées dans ce projet :")
    for lib, version in used_libraries.items():
        print(f"{lib}: {version}")

# Exécution
list_used_libraries()


Librairies utilisées dans ce projet :
pymongo: 4.10.1
jaraco.text: 3.12.1
pathlib: 1.0.1
jedi: 0.19.2
chardet: 4.0.0
polars: 0.20.31
packaging: 24.2
wrapt: 1.14.1
kagglehub: 0.3.6
traitlets: 5.14.3
astroid: 2.14.2
asttokens: 3.0.0
wcwidth: 0.2.13
ipywidgets: 8.1.5
jaraco.context: 5.3.0
decorator: 5.1.1
executing: 2.1.0
pytz: 2024.1
psutil: 6.1.1
ipykernel: 6.29.5
platformdirs: 4.3.6
skimpy: 0.0.15
pygments: 2.18.0
typeguard: 4.2.1
rich: 13.7.1
brotli: 1.0.9
requests: 2.32.3
numpy: 1.26.4
idna: 3.7
tqdm: 4.66.5
numexpr: 2.8.7
bottleneck: 1.3.7
argcomplete: 3.5.1
cryptography: 43.0.0
pickleshare: 0.7.5
zstandard: 0.23.0
six: 1.16.0
colorama: 0.4.6
pyarrow: 16.1.0
pyasn1: 0.4.8
comm: 0.2.2
bcrypt: 3.2.0
cloudpickle: 3.0.0
pandas: 2.2.3
tornado: 6.4.2
certifi: 2024.8.30
urllib3: 2.2.3
jaraco.functools: 4.0.1
parso: 0.8.4
loguru: 0.7.2
debugpy: 1.8.11
